# Chat with PDF without Langchain

## By Astra DB (Apache Cassandra), LlamaIndex, and Gradient's open-source model LLAMA2

## Install the relevant libraries

In [7]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0
!pip install llama-index-llms-gradient
!pip install llama-index-embeddings-gradient
!pip install llama-index-vector-stores-cassandra

## Import the OS and JSON modules and Cassandra

In [8]:
import os
import json
from google.colab import userdata
#generate your token from the gradient signup
os.environ['GRADIENT_ACCESS_TOKEN'] = 'Gradient token'
os.environ['GRADIENT_WORKSPACE_ID'] =  'Gradient Workspace'

In [9]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index.core import ServiceContext
from llama_index.core import set_global_service_context
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.llms.gradient import GradientBaseModelLLM
from llama_index.vector_stores.cassandra import CassandraVectorStore
import cassandra

## Connect with the Apache Cassandra vector database via SCB bundle and JSON

In [11]:
#alternative connection can also be made 

cloud_config= {
  'secure_connect_bundle': 'secure-connect-checkdatabase.zip' #downloaded from the astraDB
}

with open("token.json") as f:
    secrets = json.load(f)  #JSON file also downloaded

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("Error!")

ERROR:cassandra.connection:Closing connection <LibevConnection(133831256858656) ee59d892-2c78-42e3-b953-402093b47748-us-east1.db.astra.datastax.com:29042:92f541f4-870a-47e2-bf61-982ed96e93cb> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-051eacf61d32


## Import the LLAMA2-7b model

In [12]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

## Configure Gradient Embeddings

In [13]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

In [14]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)#cosine similarity for text analysis

set_global_service_context(service_context)

<ipython-input-14-944266470549>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


## Load the documents with whom we have to chat

In [15]:
documents = SimpleDirectoryReader("/content/docs").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 1 document(s).


## Query Index and Responses

In [16]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()

In [17]:
response = query_engine.query("What projects are there?")
print(response)

 Based on the context information provided, there are two projects:

1. Sugoi SNS - Anime Themed Social Media Website: This project was developed using the MERN tech stack (MongoDB, Express.js, Node.js, React.js, REST API) in June 2023. The project involved developing a RESTful social media website with fully functional CRUD functionality, including features for posting, editing, commenting, liking, and suggesting posts based on hashtags.
2. Monte Carlo Simulation on Molecules: This project was developed in February 2023 using C++, data structures, object-oriented programming, and Excel. The project involved developing a computational algorithm that relies on repeated random sampling to obtain numerical results, and simulating a closed system of molecules to get Lenard Jones Energy.

Therefore, the answer to the query "What projects are there?" is "Sugoi SNS and Monte Carlo Simulation on Molecules".
